In [ ]:
from tqdm.notebook import trange
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, grad, dx, ds, exp, min_value, max_value, Constant, derivative
import irksome
import icepack
from icepack2 import model

## Geometry and elevation

This notebook will demonstrate how to use the dual momentum balance model to simulate the evolution of a synthetic mountain glacier.
We'll start with a completely artificial initial glacier thickness, and then we'll spin it up to steady state.
The overall bed geometry is meant to mimic some of the features of Mt. Rainier in Washington State, but we'll produce a smaller glacier.
For many measurements, see [this volume](http://npshistory.com/publications/mora/glaciers-1963.pdf) from 1963, edited by Mark Meier and featuring many contributions by Austin Post.

Mt. Rainier has a circumference of about 150km, which works out to a radius of about 25km.
We'll use a computational domain about half that size.

In [ ]:
radius = Constant(12e3)
mesh = firedrake.UnitDiskMesh(4)
mesh.coordinates.dat.data[:] *= float(radius)

Next, we'll create some function spaces.
We'll use continuous linear finite elements for the basal elevation and discontinuous linear elements for the thickness.
In general, solving the mass balance equation for ice thickness tends to work much better with discontinuous elements.

In [ ]:
cg1 = firedrake.FiniteElement("CG", "triangle", 1)
S = firedrake.FunctionSpace(mesh, cg1)
dg1 = firedrake.FiniteElement("DG", "triangle", 1)
Q = firedrake.FunctionSpace(mesh, dg1)

Mt. Rainier is about 4km high.
We'll use a bed topography of the form $B\exp(-|x|^2/r^2)$, although this is actually a little unrealistic.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

B = Constant(4e3)
r_b = Constant(150e3 / (2 * π))
expr = B * exp(-inner(x, x) / r_b**2)
b = firedrake.Function(S).interpolate(expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_title("Bed elevation")
colors = firedrake.tripcolor(b, axes=ax)
firedrake.tricontour(b, levels=[1600.0, 2100.0], colors="black", axes=ax)
fig.colorbar(colors);

According to Meier and Post, at 1600m elevation, the surface mass balance is about -8.7 m/yr of water equivalent, and the equilibrium line altitude (where surface mass balance is zero) is at about 2100m elevation, and that the SMB must average about 0.7 m/yr above the ELA for the glacier to be in balance.
We'll use these as rough guidelines; trying to be too exact about it gives an accumulation field with a really dramatic change near the ELA that results in generalized numerical mayhem.
When we define the SMB field, we clamp the values at the top and bottom in order to avoid producing unrealistic values.
Not also how we have to convert one of the SMB values from meters of water equivalent to meters of ice equivalent by multiplying by 0.917.

In [ ]:
z_measured = Constant(1600.0)
a_measured = Constant(-0.917 * 8.7)
a_top = Constant(0.7)
z_top = Constant(4e3)
δa_δz = (a_top - a_measured) / (z_top - z_measured)
a_max = Constant(0.7)

def smb(z):
    return min_value(a_max, a_measured + δa_δz * (z - z_measured))

We'll start with a very artificial initial guess for the thickness and evolve it toward equilibrium.

In [ ]:
r_h = Constant(5e3)
H = Constant(100.0)
expr = H * firedrake.max_value(0, 1 - inner(x, x) / r_h**2)
h = firedrake.Function(Q).interpolate(expr)
h_0 = h.copy(deepcopy=True)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors);

We can then compute the initial surface elevation and mass balance.

In [ ]:
s = firedrake.Function(Q).interpolate(b + h)
a = firedrake.Function(Q).interpolate(smb(s))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(a, vmin=-0.7, axes=ax)
fig.colorbar(colors);

## Initial velocity computation

In order to calculate the material properties of the glacier, we'll assume that it's roughly temperate, which isn't too far off from the truth.

In [ ]:
from icepack2.constants import gravity, ice_density, glen_flow_law
A = icepack.rate_factor(Constant(273.15))

We'll use the shallow ice approximation, which diagnoses the velocity at a point in a purely local fashion using the thickness and surface slope, to initialize the ice velocity.

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, cg1)

u = firedrake.Function(V)
v = firedrake.TestFunction(V)

ρ_I = Constant(ice_density)
g = Constant(gravity)
n = Constant(glen_flow_law)

P = ρ_I * g * h
S_n = inner(grad(s), grad(s))**((n - 1) / 2)
u_shear = -2 * A * P ** n / (n + 2) * h * S_n * grad(s)
F = inner(u - u_shear, v) * dx

solver_params = {"snes_type": "ksponly", "ksp_type": "gmres"}
fc_params = {"quadrature_degree": 6}
params = {"solver_parameters": solver_params, "form_compiler_parameters": fc_params}
firedrake.solve(F == 0, u, **params)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(u, axes=ax)
fig.colorbar(colors);

Now we'll get to the real modeling.
To proceed, we have to make function spaces to represent the membrane and basal stresses and create a variable that stores the velocity and stresses.
Here we use piecewise constant finite elements for the stresses.
In general, if you use $CG(k)$ elements for the velocity, you need to use $DG(k - 1)$ elements for the stresses.

In [ ]:
dg0 = firedrake.FiniteElement("DG", "triangle", 0)
Σ = firedrake.TensorFunctionSpace(mesh, dg0, symmetry=True)
T = firedrake.VectorFunctionSpace(mesh, cg1)
Z = V * Σ * T
z = firedrake.Function(Z)

z.sub(0).assign(u);

This is a subtle but important point.
But we'll also want to regularize the momentum balance model a bit by pretending as if it were linear.
We chose some value of the ice fluidity $A$, which is fine, but remember that $A$ has units of strain rate $\times$ stress${}^{-n}$ where $n \approx 3$ is the Glen flow law exponent.
If we only change $n$ to 1 in the parts of the equation determining the constitutive relation but do nothing to $A$, the physical units that we use in the regularization will make no sense.
So what we do below is pick some arbitrary stress scale $\tau_c$, which here I'm taking to be 0.1 MPa or 100 kPa.
We're then choosing a strain rate scale $\dot\varepsilon_c$ so that $A = \dot\varepsilon_c \cdot\tau_c^{-n}$.
If we want to pretend like ice flow were linear, we could produce the right constitutive relation by then using the flow law coefficient $A' = \dot\varepsilon_c\cdot\tau_c^{-1}$ and all the units would make sense.

In [ ]:
τ_c = Constant(0.1)
ε_c = Constant(A * τ_c ** n)

Now we're at the really important part.
To make the SSA model effectively do SIA, what we need is that, in the sliding law
$$K|\tau|^{m - 1}\tau = -u,$$
we take
$$K = \frac{hA}{n + 2}.$$
Now in order to make our regularization approach work, we also define the sliding coefficient as
$$K = u_c / \tau_c^m$$
where $\tau_c$ is a reference stress and $u_c$ a reference speed.
We're using the same reference stress for the flow and sliding laws of $\tau_c = $ 100 kPa, so we can then compute the reference speed as $u_c = K\tau_c^m$.
We can then add more sliding on top of the effective deformation that we use to mimic SIA.
To add some extra real sliding, we've increased the critical speed $u_c$ by a factor of 100 m/yr.

In [ ]:
K = h * A / (n + 2)
U_c = Constant(100.0)
u_c = K * τ_c ** n + U_c

In [ ]:
rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c ** n,
    "sliding_exponent": n,
    "sliding_coefficient": u_c / τ_c ** n,
}

u, M, τ = firedrake.split(z)
fields = {
    "velocity": u,
    "membrane_stress": M,
    "basal_stress": τ,
    "thickness": h,
    "surface": s,
}

v, N, σ = firedrake.TestFunctions(Z)

fns = [
    (model.variational.flow_law, N),
    (model.variational.friction_law, σ),
    (model.variational.momentum_balance, v),
]

Now we'll create the parameters that we need for the solver, including what quadrature degree and solution strategy to use.

In [ ]:
degree = 1
qdegree = max(8, degree ** glen_flow_law)
pparams = {"form_compiler_parameters": {"quadrature_degree": qdegree}}

sparams = {
    "solver_parameters": {
        "snes_monitor": ":rainier-output-monolothic.txt",
        "snes_type": "newtonls",
        "snes_max_it": 200,
        "snes_linesearch_type": "nleqerr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}

First, we'll do an initial solve with a linear rheology.
Then we'll gradually step the flow law exponent from 1.0 up to 3.0.

In [ ]:
terms = [fn(**fields, **rheology, test_function=ϕ) for fn, ϕ in fns]
terms[0] = firedrake.replace(terms[0], {h: H})
F = sum(terms)
momentum_problem = firedrake.NonlinearVariationalProblem(F, z, **pparams)
momentum_solver = firedrake.NonlinearVariationalSolver(momentum_problem, **sparams)

Now we can see our initial value of the velocity.
Note how it's a different magnitude but similar shape to the SIA velocity.

In [ ]:
num_continuation_steps = 5
for exponent in np.linspace(1.0, 3.0, num_continuation_steps):
    n.assign(exponent)
    momentum_solver.solve()

In [ ]:
u_init, M_init, τ_init = z.subfunctions

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(u_init, axes=axes)
fig.colorbar(colors);

## Forward modeling

Let's zoom in and look at where the ELA falls.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(u_init, axes=axes)
firedrake.triplot(mesh, axes=axes)
firedrake.tricontour(a, levels=[0, 1], colors="tab:orange", axes=axes)
fig.colorbar(colors);

Now make an even bigger function space, God help us all.
Pack in the initial values of the velocity, membrane stress, and basal stress that we computed before.

In [ ]:
W = V * Σ * T * Q
w = firedrake.Function(W)
w.sub(0).assign(u_init)
w.sub(1).assign(M_init)
w.sub(2).assign(τ_init)
w.sub(3).assign(h);

Next, form the momentum balance equation again.

In [ ]:
u, M, τ, h = firedrake.split(w)
v, N, σ, η = firedrake.TestFunctions(W)

fns = [
    (model.variational.flow_law, N),
    (model.variational.friction_law, σ),
    (model.variational.momentum_balance, v),
]

fields = {
    "velocity": u,
    "membrane_stress": M,
    "basal_stress": τ,
    "thickness": h,
    "surface": s,
}

terms = [fn(**fields, **rheology, test_function=ϕ) for fn, ϕ in fns]
terms[0] = firedrake.replace(terms[0], {h: H})
F_momentum = sum(terms)
F_mass = model.mass_balance(thickness=h, velocity=u, accumulation=a, test_function=η)

F = F_momentum + F_mass

In [ ]:
tableau = irksome.BackwardEuler()
t = Constant(0.0)
dt = Constant(1.0 / 12)
solver = irksome.TimeStepper(F, tableau, t, dt, w, **sparams)

The good part: run the model for 500 years of simulation time.

In [ ]:
us = [w.subfunctions[0].copy(deepcopy=True)]
hs = [w.subfunctions[3].copy(deepcopy=True)]

final_time = 500.0
num_steps = int(final_time / float(dt))
for step in trange(num_steps):
    solver.advance()
    h = w.subfunctions[3]
    w.subfunctions[3].interpolate(max_value(0, h))
    s.interpolate(b + h)
    a.interpolate(smb(s))

    us.append(w.subfunctions[0].copy(deepcopy=True))
    hs.append(w.subfunctions[3].copy(deepcopy=True))

Now let's make a movie of the evolution towards steady state.
The contour shows the final value of the ELA.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(hs[-1], num_sample_points=1, shading="flat", axes=axes)
firedrake.triplot(mesh, axes=axes)
firedrake.tricontour(a, levels=[0, 1], colors="tab:orange", axes=axes)
fig.colorbar(colors);

In [ ]:
u, M, τ = z.subfunctions

fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(us[-1], num_sample_points=1, shading="flat", axes=axes)
firedrake.triplot(mesh, axes=axes)
firedrake.tricontour(a, levels=[0, 1], colors="tab:orange", axes=axes)
fig.colorbar(colors);

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(hs[0], vmax=130.0, num_sample_points=4, axes=axes)
fig.colorbar(colors)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(h):
    colors.set_array(fn_plotter(h))

animation = FuncAnimation(fig, animate, hs, interval=1e3/60)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())